# Identifying Port Calls in the PIERS BoL Data

### Problem Statement:

The PIERS data do not have clean arrival or departure dates; instead, cargo imported from a given shipping vessel at a given port is spread over multiple dates, often spanning several days up to several weeks, even though the vessels spend only a matter of hours at any given port call. The reason for this spread is not precisely known at this time, but given the underlying data come from US Customs, we expect it may be related to the time required to process the individual bills. 

Therefore we need to identify clusters of dates that indicate a unique port call for each vessel, with the goal of assigning the earliest date in the cluster as the actual arrival/departure date for imports/exports. 

Potential uses of this information include analyzing frequency of service at each port, identification of individual voyages, and many others. 

In [ ]:
#prelims
import pandas as pd #v2.1.3
import polars as pl #v0.20.7
import plotly_express as px #v0.4.1 
import missingno as msno #v0.5.2
import datetime as dt

#display settings
pd.set_option('display.max_columns', None)

#enable string cache for polars categoricals
pl.enable_string_cache()